## Pipeline Script 
This script 

### Import Packages

In [35]:
import os
import glob
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import pearsonr
from scipy.stats import linregress
import seaborn as sns
import matplotlib.pyplot as plt
import re
import shutil

### Define paths and variables

In [36]:
# Set paths
fcpath = "/Users/pecsok/Desktop/ImageData/PMACS_remote/data/fmri/postprocessed/3T/xcp_d"
outpath = "~/Desktop/ImageData/PMACS_remote/analysis/postprocessing/"
clinpath = "~/Desktop/ImageData/PMACS_remote/data/clinical"
cestpath = "~/Desktop/ImageData/PMACS_remote/data/cest"

# Choose what to analyse
#networks = ["Cont", "Default", "DorsAttn", "Vis", "SalVentAttn", "SomMot", "Limbic"] 
networks = ["THA","CAU"] 
#btwn_ntwks = ["Cont_btwn", "Default_btwn", "DorsAttn_btwn", "Vis_btwn", "SalVentAttn_btwn", "SomMot_btwn", "Limbic_btwn"] 
btwn_ntwks = ["THA_btwn","CAU_btwn"] 
#all_ntwks = ["Cont_all", "Default_all", "DorsAttn_all", "Vis_all", "SalVentAttn_all", "SomMot_all", "Limbic_all"] 
all_ntwks = ["THA_all","CAU_all"] 
networkscest = ["Right Thalamus","Right Caudate"] 


CESTnetworks = ["avgCEST_THA", "ctCEST_THA", "avgCEST_CAU", "ctCEST_CAU"]

CNB_scores = ["tap_tot"]
CNB_valids = ["tap_valid"] 
diag_scores = ["hstatus","age"]
demo_scores = ["sex", "race","ethnic","dateDiff"]
diag_details = ["axis1_desc1", "axis1_desc2", "axis1_desc3","axis1_desc4","axis1_desc5", "axis1_desc6"]


# Make dataframe based on metrics of interest
grp_df = pd.DataFrame(columns = ["BBLID"] + ["Session"] + ["SCANID_CEST"] + demo_scores + networks + btwn_ntwks + CESTnetworks + CNB_scores + diag_scores)
diag_df = pd.DataFrame(columns = ["BBLID"] + ["Session"] + ["SCANID_CEST"] + diag_scores + diag_details)
print(grp_df)

# Initialize empty lists and vars
bblids = []
sesids = []
gluseses = []

# Import group dataframes and set indices
subjlist = pd.read_csv("~/Desktop/ImageData/PMACS_remote/data/subject_list_031124.csv", sep=',') 
subjlist = subjlist.dropna(subset=['BBLID'])
subjlist['BBLID'] = subjlist['BBLID'].astype(int)
subjlist['SCANID_CEST'] = subjlist['SCANID_CEST'].astype(int)
cnbmat = pd.read_csv(clinpath + "/maggie_datarequest_fullcnb-2.csv", sep=',') 
diagmat = pd.read_csv(clinpath + "/maggie_datarequest_diagnosis-2.csv", sep=',')
demomat = pd.read_csv(clinpath + "/maggie_datarequest_demographics-2.csv", sep=',')
cestmat = pd.read_csv(cestpath + "/compiled_outputs/compiled_all_subs_GluCEST_sub_rois_UNI.csv", sep=',')
# cestmat = pd.read_csv(clinpath + "/demographics.csv", sep='\t') add grp CEST map here
cnbmat.set_index('bblid', inplace = True)
diagmat.set_index('bblid', inplace = True)
demomat.set_index('bblid', inplace = True)
cestmat.set_index('Subject', inplace = True)

# Set up renaming dictionary for CEST df
schaefer_indices = pd.read_csv('~/Desktop/ImageData/PMACS_remote/github/glucest-rsfmri/Schaefer2018_100Parcels_17Networks_order_FSLMNI152_2mm.Centroid_RAS.csv', sep=',') # Load the CSV with the mapping of numbers to labels
schaefer_dict = dict(zip(schaefer_indices['ROI Label'], schaefer_indices['ROI Name']))

#print(subjlist)

Empty DataFrame
Columns: [BBLID, Session, SCANID_CEST, sex, race, ethnic, dateDiff, THA, CAU, THA_btwn, CAU_btwn, avgCEST_THA, ctCEST_THA, avgCEST_CAU, ctCEST_CAU, tap_tot, hstatus, age]
Index: []


### Choose which modules to run

In [37]:
runfcon = True
runCNB = True
rundiag = True
rundemo = True
runcest = True
run_grpanalysis = True

## Stage 1: Create Group Data Frame
### Import data, loop through subjects, and establish file paths


FIX THIS ERROR:
/var/folders/ls/hy_z7hgd4_13km3h7j84vqh40000gp/T/ipykernel_77945/3898733492.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'PSY' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grp_df.loc[grp_df['BBLID'].astype(str) == bblid, grp_df.columns == diag_score] = diagnosis

In [38]:
print(grp_df)


Empty DataFrame
Columns: [BBLID, Session, SCANID_CEST, sex, race, ethnic, dateDiff, THA, CAU, THA_btwn, CAU_btwn, avgCEST_THA, ctCEST_THA, avgCEST_CAU, ctCEST_CAU, tap_tot, hstatus, age]
Index: []


In [39]:
# Generates list of all file names
folder_names = [folder for folder in glob.glob(os.path.join(fcpath, "*")) if os.path.isdir(folder)]
#subje

# Loop through subjects
for subj_path in folder_names: # loop through all rows of the spreadsheet
    if "sub" in subj_path:
        # Extract bblid id:
        bblid = subj_path.split('-')[1]
        print("Processing subject " + bblid)
        # Extract session id: 
        items = os.listdir(subj_path)
        ses_folder = [item for item in items if item.startswith("ses")]
        ses = ses_folder[0].split('-')[1]
        ses_path = os.path.join(fcpath, subj_path, ses_folder[0]) # full path to session
        if bblid in subjlist['BBLID'].astype(str).values:
            gluses = subjlist.loc[subjlist['BBLID'].astype(str) == bblid, 'SCANID_CEST'].values[0].astype(str) #.
         # Add to running list of IDs grp analysis later:
        bblids.append(bblid)
        sesids.append(ses)
        gluseses.append(gluses)
        # Start new row in grp_df for this subject:
        ids = [bblid, ses, gluses]  # Values for the first two columns
        grp_df.loc[len(grp_df)] = ids + [float('nan')] * (len(grp_df.columns) - len(ids))
        diag_df.loc[len(diag_df)] = ids + [float('nan')] * (len(diag_df.columns) - len(ids))   

        # Run a subset of subjs or exclude specific subjs.
        if bblid != "20902" and bblid != "93242"  and bblid != "20754" and bblid != "127065":
            ##################################################################################################
            ## FC
            ##################################################################################################
            if runfcon:
                os.path.join(fcpath, "sub-" + bblid, "ses-" + ses)
                ses_path = os.path.join(fcpath, subj_path, ses_folder[0]) # full path to session
                fcmat_glob = f"{ses_path}/func/*subcortical_measure-pearsoncorrelation_conmat.tsv"
                #fcmat_glob = f"{ses_path}/func/*Schaefer117_measure-pearsoncorrelation_conmat.tsv"
                if os.path.isfile(glob.glob(fcmat_glob)[0]):
                    fcmat = pd.read_csv(glob.glob(fcmat_glob)[0], sep='\t') # read in fcmat
                    fcmat.set_index('Node', inplace = True)
                    #print(fcmat)
                    # Loop through the networks
                    for i in range(len(networks)):
                        network = networks[i]
                        btwn_ntwk = btwn_ntwks[i]
                        all_ntwk = all_ntwks[i]
                        print("Running " + network + " fcon")
                        
                        # Extract within-network FC
                        mask = np.triu(np.ones_like(fcmat, dtype=bool))
                        fcmat = fcmat.mask(mask)
                        np.fill_diagonal(fcmat.values, 1)
                        #print(fcmat.index)
                        filtered_rows = fcmat.index.str.contains(network) & fcmat.index.str.contains("rh")
                        filtered_columns = fcmat.columns.str.contains(network) & fcmat.columns.str.contains("rh")
                        network_fc = fcmat.loc[filtered_rows, filtered_columns]
                        #print(network_fc)
                        # Calculate avg network fc and add value to proper column in grp_df
                        grp_df.loc[len(grp_df)-1, network] = np.nanmean(network_fc.values)

                        # Calculate between-network FC
                        fcmat_all = fcmat.combine_first(fcmat.T)
                        btwn_fc = fcmat.loc[filtered_rows, fcmat.columns[~filtered_columns]]
                        # Calculate avg network fc and add value to proper column in grp_df
                        grp_df.loc[grp_df['BBLID'] == bblid, btwn_ntwk] = np.nanmean(btwn_fc)

                        # Calculate all FC
                        np.fill_diagonal(fcmat.values, np.nan)
                        all_fc = fcmat.loc[filtered_rows, :]
                        # Calculate avg network fc and add value to proper column in grp_df
                        grp_df.loc[grp_df['BBLID'] == bblid, all_ntwk] = np.nanmean(all_fc)
            
                        # Select rows and columns corresponding to the network
                    #    np.fill_diagonal(fcmat.values, np.nan)
                    #    fcmat['avg_fc'] = fcmat.mean(axis=1, skipna=True)
                    #    fc_all = fcmat['avg_fc'].tail(50).values
                       # last50 = pd.DataFrame(fc_last50).T
                       # last50 = last50.rename(columns={i: str(i+1) for i in range(50)})
                       # fc_parcelmat.iloc[fc_parcelmat['BBLID'] == bblid, 8:] = fc_last50 #fc_parcelmat.index == bblid

            
            ##################################################################################################
            ## CNB
            ##################################################################################################
            if runCNB:
                # Loop through the CNB scores
                for i in range(len(CNB_scores)):
                    CNB_score = CNB_scores[i]
                    CNB_valid = CNB_valids[i]
                    # Select score of interest & validity of that score
                    scores = cnbmat[CNB_score]
                    if int(bblid) in scores.index:
                        score = scores[int(bblid)]
                        valids = cnbmat[CNB_valid]
                        valid = str(valids[int(bblid)])
                        # If score was valid, add to grp_df
                        if 'V' in valid: 
                            grp_df.loc[grp_df['BBLID'] == bblid, grp_df.columns == CNB_score] = score 
            ##################################################################################################
            ## Diagnosis
            ##################################################################################################
            if rundiag:
                # Add hstatus
                for i in range(len(diag_scores)):
                    diag_score = diag_scores[i]
                    # Select score of interest and add to grp_df
                    diagnoses = diagmat[diag_score]
                    if int(bblid) in diagnoses.index:
                        diagnosis = diagnoses[int(bblid)]
                        grp_df.loc[grp_df['BBLID'].astype(str) == bblid, grp_df.columns == diag_score] = diagnosis 
                        diag_df.loc[diag_df['BBLID'].astype(str) == bblid, diag_df.columns == diag_score] = diagnosis 
                    else:
                        diagnosis = "Unknown"
                        grp_df.loc[grp_df['BBLID'].astype(str) == bblid, grp_df.columns == diag_score] = diagnosis 
                        diag_df.loc[diag_df['BBLID'].astype(str) == bblid, diag_df.columns == diag_score] = diagnosis
     
                # Loop through diag scores
                for i in range(len(diag_details)):
                    diag_detail = diag_details[i]
                    # Select score of interest and add to grp_df
                    comorbidities = diagmat[diag_detail]
                    if int(bblid) in comorbidities.index:
                        comorbidity = comorbidities[int(bblid)]
                       # print(comorbidity)
                        diag_df.loc[diag_df['BBLID'].astype(str) == bblid, diag_df.columns == diag_detail] = comorbidity 
                    else:
                        comorbidity = "Unknown"
                        diag_df.loc[diag_df['BBLID'].astype(str) == bblid, diag_df.columns == diag_detail] = comorbidity
            ##################################################################################################
            ## Demographics
            ##################################################################################################
            if rundemo:
                # Loop through the CNB scores
                for i in range(len(demo_scores)):
                    demo_score = demo_scores[i]
                    # Select metric of interest
                    scores = demomat[demo_score]
                    if int(bblid) in scores.index:
                        score = scores[int(bblid)]
                        # Add to grp_df
                        grp_df.loc[grp_df['BBLID'] == bblid, grp_df.columns == demo_score] = score 
            ##################################################################################################
            ## CEST
            ##################################################################################################
            if runcest and bblid != "88760": 
                print("Processing " + bblid + "'s CEST data'")
                # Extract Glu Session ID
                if bblid in subjlist['BBLID'].astype(str).values:
                    gluses = subjlist.loc[subjlist['BBLID'].astype(str) == bblid, 'SCANID_CEST'].values[0].astype(str) #.
                    cestid = bblid + "_" + gluses
                    print(cestid)
                    # Import data
                    for i in range(len(networks)):
                        network = networks[i]
                        print(network)
                        networkcest = networkscest[i]
                        col_name = "avgCEST_" + network # for grp_df
                        ct_name = "ctCEST_" + network # for grp_df
                        col_cestmat = networkcest + " NZMean" # for CEST output 
                        print(col_cestmat)
                        ct_cestmat = networkcest + " NZcount"
                        if str(cestid) in cestmat.index:    
                            print("yes")
                            cest_avgs = cestmat[col_cestmat]
                            cest_avg = cest_avgs[str(cestid)]
                            print(cest_avg)
                            cest_cts = cestmat[ct_cestmat]
                            cest_ct = cest_cts[str(cestid)]
                            grp_df.loc[grp_df['BBLID'].astype(str) == bblid, grp_df.columns == col_name] = cest_avg 
                            grp_df.loc[grp_df['BBLID'].astype(str) == bblid, grp_df.columns == ct_name] = cest_ct                            

print(grp_df)
# sum_of_mean_columns now contains the sum of values in columns with "Mean" in the column name.

Processing subject 117847
Running THA fcon
Running CAU fcon
Processing 117847's CEST data'
117847_12740
THA
Right Thalamus NZMean
CAU
Right Caudate NZMean
Processing subject 20645


/var/folders/kk/w6xmtt2d55xfbxvb6wcqhq580000gp/T/ipykernel_55686/1942517382.py:109: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'O' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grp_df.loc[grp_df['BBLID'].astype(str) == bblid, grp_df.columns == diag_score] = diagnosis
/var/folders/kk/w6xmtt2d55xfbxvb6wcqhq580000gp/T/ipykernel_55686/1942517382.py:110: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'O' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  diag_df.loc[diag_df['BBLID'].astype(str) == bblid, diag_df.columns == diag_score] = diagnosis
/var/folders/kk/w6xmtt2d55xfbxvb6wcqhq580000gp/T/ipykernel_55686/1942517382.py:124: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pan

Running THA fcon
Running CAU fcon
Processing 20645's CEST data'
20645_11260
THA
Right Thalamus NZMean
yes
7.487255
CAU
Right Caudate NZMean
yes
nan
Processing subject 128865
Running THA fcon
Running CAU fcon
Processing 128865's CEST data'
128865_12325
THA
Right Thalamus NZMean
yes
7.828702
CAU
Right Caudate NZMean
yes
nan
Processing subject 120217


/var/folders/kk/w6xmtt2d55xfbxvb6wcqhq580000gp/T/ipykernel_55686/1942517382.py:124: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Major Depressive Disorder, Recurrent, In Partial Remission' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  diag_df.loc[diag_df['BBLID'].astype(str) == bblid, diag_df.columns == diag_detail] = comorbidity
/var/folders/kk/w6xmtt2d55xfbxvb6wcqhq580000gp/T/ipykernel_55686/1942517382.py:124: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Cannabis Dependence' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  diag_df.loc[diag_df['BBLID'].astype(str) == bblid, diag_df.columns == diag_detail] = comorbidity
/var/folders/kk/w6xmtt2d55xfbxvb6wcqhq580000gp/T/ipykernel_55686/1942517382.py:124: FutureWarning: Setting an item of

Running THA fcon
Running CAU fcon
Processing 120217's CEST data'
120217_10722
THA
Right Thalamus NZMean
yes
5.451816
CAU
Right Caudate NZMean
yes
nan
Processing subject 21118
Running THA fcon
Running CAU fcon
Processing 21118's CEST data'
21118_12784
THA
Right Thalamus NZMean
yes
6.911625
CAU
Right Caudate NZMean
yes
7.298252
Processing subject 125511
Running THA fcon
Running CAU fcon
Processing 125511's CEST data'
125511_10906
THA
Right Thalamus NZMean
yes
7.447943
CAU
Right Caudate NZMean
yes
nan
Processing subject 105176
Running THA fcon
Running CAU fcon
Processing 105176's CEST data'
Processing subject 132179
Running THA fcon
Running CAU fcon
Processing 132179's CEST data'
132179_10760
THA
Right Thalamus NZMean
yes
6.914718
CAU
Right Caudate NZMean
yes
nan
Processing subject 15305
Running THA fcon
Running CAU fcon
Processing 15305's CEST data'
Processing subject 17621
Running THA fcon
Running CAU fcon
Processing 17621's CEST data'
Processing subject 98831
Running THA fcon
Running C

/var/folders/kk/w6xmtt2d55xfbxvb6wcqhq580000gp/T/ipykernel_55686/1942517382.py:113: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grp_df.loc[grp_df['BBLID'].astype(str) == bblid, grp_df.columns == diag_score] = diagnosis
/var/folders/kk/w6xmtt2d55xfbxvb6wcqhq580000gp/T/ipykernel_55686/1942517382.py:114: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  diag_df.loc[diag_df['BBLID'].astype(str) == bblid, diag_df.columns == diag_score] = diagnosis
/var/folders/kk/w6xmtt2d55xfbxvb6wcqhq580000gp/T/ipykernel_55686/1942517382.py:127: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future ve

Running THA fcon
Running CAU fcon
Processing 135277's CEST data'
135277_12808
THA
Right Thalamus NZMean
yes
6.583712
CAU
Right Caudate NZMean
yes
7.232149
Processing subject 117397
Running THA fcon
Running CAU fcon
Processing 117397's CEST data'
117397_10686
THA
Right Thalamus NZMean
yes
13.398516
CAU
Right Caudate NZMean
yes
7.900541
Processing subject 93274
Running THA fcon
Running CAU fcon
Processing 93274's CEST data'
93274_10765
THA
Right Thalamus NZMean
yes
7.887604
CAU
Right Caudate NZMean
yes
8.194631
Processing subject 20902
Processing subject 128259
Running THA fcon
Running CAU fcon
Processing 128259's CEST data'
128259_12837
THA
Right Thalamus NZMean
yes
8.08732
CAU
Right Caudate NZMean
yes
8.992452
Processing subject 93242
Processing subject 106880
Running THA fcon
Running CAU fcon
Processing 106880's CEST data'
106880_10699
THA
Right Thalamus NZMean
yes
8.371712
CAU
Right Caudate NZMean
yes
8.817147
Processing subject 131384
Running THA fcon
Running CAU fcon
Processing 131

In [63]:
print(grp_df)

     BBLID  Session SCANID_CEST  sex  race  ethnic  dateDiff      Cont  \
0   117847    12564       12740  1.0   1.0     2.0      18.0  0.083760   
1    20645    11274       11260  1.0   2.0     2.0       0.0  0.266631   
2   128865    12165       12325  1.0   1.0     2.0      49.0  0.080022   
3   120217    10702       10722  2.0   1.0     2.0       8.0  0.170671   
4    21118    12258       12784  2.0   1.0     2.0       0.0  0.155164   
..     ...      ...         ...  ...   ...     ...       ...       ...   
87  127935  motive2       12101  1.0   2.0     2.0      35.0  0.127090   
88   94703    11928       12082  2.0   2.0     2.0      13.0  0.124214   
89   91335     9346       12082  1.0   2.0     2.0     113.0  0.138340   
90   89095     9972       11100  2.0   1.0     2.0       0.0  0.169108   
91   20082  motive2       11821  2.0   1.0     2.0      18.0  0.033836   

     Default  DorsAttn  ...  ctCEST_Vis  avgCEST_Limbic  ctCEST_Limbic  \
0   0.181050  0.321492  ...         N

In [25]:
print(grp_df['avgCEST_SomMot'].isna().sum())

20


In [40]:
grp_df.to_csv('grp_df_3T_SOBP_wba_BG.csv', index=True)
#diag_df.to_csv('diag_df_3T_SOBP_wba.csv', index=True)

In [64]:
grp_df.to_csv('grp_df_3T.csv', index=True)
diag_df.to_csv('diag_df_3T.csv', index=True)

In [13]:
#grp_df = pd.read_csv('012624_grp_df_3T.csv')
pd.set_option('display.max_rows', None)


In [34]:
!jupyter nbconvert --to html motor_pipeline.ipynb --output motor_pipeline_3T.html

[NbConvertApp] WARNING | Config option `kernel_spec_manager_class` not recognized by `NbConvertApp`.
[NbConvertApp] Converting notebook motor_pipeline.ipynb to html
[NbConvertApp] Writing 1146801 bytes to motor_pipeline_3T.html
